## Netflix Subscription Forecasting
This project aim estimate the expected number of subscriber of Netflix in the given period through ARIMA model and present in-depth analysis.

### 1. Data Processing

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objs as go
import plotly.express as px
import plotly.io as pio
import sys
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

In [13]:
df = pd.read_csv('Data/Netflix-Subscriptions.csv')
df.head()

,Time Period,Subscribers
0,01/04/2013,34240000
1,01/07/2013,35640000
2,01/10/2013,38010000
3,01/01/2014,41430000
4,01/04/2014,46130000


The dataset contain netflix quarterly subscription data from 2013 to 2023. Before first step analysis, we need to convert the time period data from str to datetime format.

In [14]:
df['Date'] = pd.to_datetime(df['Time Period'],format='%d/%m/%Y')
df.head()

,Time Period,Subscribers,Date
0,01/04/2013,34240000,2013-04-01
1,01/07/2013,35640000,2013-07-01
2,01/10/2013,38010000,2013-10-01
3,01/01/2014,41430000,2014-01-01
4,01/04/2014,46130000,2014-04-01


In [15]:

fig = go.Figure()
fig.add_trace(go.Scatter(x=df['Date'],
                         y=df['Subscribers'],
                         mode='lines', name='Subscribers'))
fig.update_layout(title='Netflix Quarterly Subscriptions Growth',
                  xaxis_title='Date',
                  yaxis_title='Netflix Subscriptions')
fig.show()